In [1]:
from random import randint, random
import collections
import os
import shutil

import numpy as np
import math
import time
from collections import deque
import torch as T
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import sklearn.neighbors
import sklearn.metrics
import sklearn.mixture
import matplotlib.pyplot as plt
import PIL
from nvidia.dali.plugin.pytorch import DALIGenericIterator

from imageComponent import *
from remote_read_write import scp_read_wrapper, scp_write_wrapper
scp_Tload = scp_read_wrapper(T.load, 'ly_1112103017@172.16.35.121', 30907, '/public/ly/zyz/cluster_id_rsa', '/public/ly/zyz/imageCL/tmp')
scp_Tsave = scp_write_wrapper(T.save, 'ly_1112103017@172.16.35.121', 30907, '/public/ly/zyz/cluster_id_rsa', '/public/ly/zyz/imageCL/tmp')

dali_device = 2
device = 'cuda:1'
T.backends.cudnn.benchmark = True
T.backends.cudnn.deterministic = True
use_amp = False

/public/ly/anaconda3/envs/zyz/lib/python3.10/site-packages/nvidia/dali/backend.py:46: Warning: DALI support for Python 3.10 is experimental and some functionalities may not work.
  deprecation_warning("DALI support for Python 3.10 is experimental and some functionalities "


In [2]:
model = ResnetCifar(18)
root = '/public/home/ly_1112103017/zyz/imageCL/ckpt/CLCifarRes18'
epoch = 1200
smodel = copy.deepcopy(model)
sroot = '/public/home/ly_1112103017/zyz/imageCL/ckpt/SCifarRes18'
sepoch = 120

In [3]:
trainfolder = '/public/ly/zyz/imageCL/dataset/cifar10-image/train'
rawtrainfolder = '/public/ly/zyz/imageCL/dataset/cifar10-image/train'
valfolder = '/public/ly/zyz/imageCL/dataset/cifar10-image/test'
size = (32, 32)
batch_size = 3125

In [4]:
files = ['%s/%s' %(i, j) for i in os.listdir(trainfolder) for j in os.listdir('%s/%s' %(trainfolder, i))]

In [5]:
rawtrainpipe = DALIHadhValdImageFolders(root=trainfolder, files=files, size=size, batch_size=batch_size,
                                        num_threads=6, device_id=dali_device)
rawtrainloader = dali.plugin.pytorch.DALIGenericIterator([rawtrainpipe], reader_name ='reader', output_map=['image', 'label'],
                                                    last_batch_policy=dali.plugin.base_iterator.LastBatchPolicy.PARTIAL)
#valpipe = DALIValdImageFolders(root=valfolder, size=size, batch_size=batch_size, num_threads=4, device_id=dali_device)
#valloader = dali.plugin.pytorch.DALIGenericIterator([valpipe], reader_name='reader', output_map=['image', 'label'],
#                                                    last_batch_policy=dali.plugin.base_iterator.LastBatchPolicy.PARTIAL)

In [6]:
with T.inference_mode():
    with T.autocast(device_type='cuda', dtype=T.float16, enabled=use_amp):
        model.load_state_dict(scp_Tload('%s/%04d.pth' %(root, epoch), map_location=device))
        model = model.to(device, memory_format=T.channels_last)
        X, y = extract_feature(rawtrainloader, model, device, normalize=True, use_amp=use_amp)
        
        smodel.load_state_dict(scp_Tload('%s/%04d.pth' %(sroot, sepoch), map_location=device))
        smodel = smodel.to(device, memory_format=T.channels_last)
        sX, sy = extract_feature(rawtrainloader, smodel, device, normalize=True, use_amp=use_amp)
del model, smodel
class_y = T.tensor([int(files[i][0]) for i in y]).to(device)

In [7]:
T.save((X, y, sX, sy, class_y), '/public/ly/zyz/imageCL/feature/Res18.pth')

In [ ]:
nX = X - X.min()

In [ ]:
nX /= nX.max()

In [ ]:
with T.inference_mode():
    tnX = tsne(nX[:5000], 2, 50, perplexity=4900)

In [ ]:
tnX = tnX.cpu().numpy()
class_y = class_y[:5000].cpu().numpy()

In [ ]:
for i in range(10):
    idx = class_y == i
    plt.scatter(tnX[idx, 0], tnX[idx, 1], alpha=0.6)

sX.cpu(), sy.cpu()
T.cuda.empty_cache()

with T.inference_mode():
    cos = cos_similarity(X)
    idx = (class_y[:, None] == class_y[None]).int()
    idx *= 10
    cos -= idx
    #idx = list(range(len(y)))
    #cos[idx, idx] = -10000
    cos = cos.flatten()
    tmp = T.max(cos, dim=0)
    del cos, idx
T.cuda.empty_cache()

k = files[y[tmp[1] // 50000]]
v = files[y[tmp[1] % 50000]]

os.makedirs('%s' %root[-3:])
shutil.copy('%s/%s' %(trainfolder, k), '%s/%fk.png' %(root[-3:], tmp[0].item()))
shutil.copy('%s/%s' %(trainfolder, v), '%s/%fv.png' %(root[-3:], tmp[0].item()))

with T.inference_mode():
    sX.to(device), sy.to(device)
    scos = (sX[y[tmp[1] // 50000]] * sX[y[tmp[1] % 50000]]).sum()

In [12]:
scos

tensor(-0.0698, device='cuda:3')